# NoSQL Database Explorations for Spatial Analysis

Alison Link | MSI Informatics Interest Group | 13 March 2019

<img src="img/nosql.png" alt="noSQL logo" width="600"/>

# What do we mean by 'NoSQL'?

NoSQL databases don't like to be labeled!

> "...when 'NoSQL' is applied to a database, it refers to an **ill-defined set** of mostly open-source databases, mostly developed in the early 21st century, and mostly not using SQL." (_NoSQL Distilled_, p. 11)

> "To summarize the summary, **it neither makes sense to precisely define NoSQL**, nor to simply say that Elasticsearch is a "document store"-type NoSQL-database." ([ElasticSearch as a NoSQL Database](https://www.elastic.co/blog/found-elasticsearch-as-nosql))




* **Document** --> MongoDB, Elasticsearch

* **Graph** --> Neo4j

* **Key-Value** --> Riak

* **Column-Family** --> Cassandra

(_NoSQL Distilled_, p. xvii)

<img src="img/documents.png" alt="document icon" width="200" style="float:left;padding:20px"/>

<img src="img/graph.png" alt="graph icon" width="300" style="float:left;padding:20px"/>

# Twitter data objects

https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html

```json
{
  "created_at": "Thu Apr 06 15:24:15 +0000 2017",
  "id_str": "850006245121695744",
  "text": "1\/ Today we\u2019re sharing our vision for the future of the Twitter API platform!\nhttps:\/\/t.co\/XweGngmxlP",
  "user": {
    "id": 2244994945,
    "name": "Twitter Dev",
    "screen_name": "TwitterDev",
    "location": "Internet",
    "url": "https:\/\/dev.twitter.com\/",
    "description": "Your official source for Twitter Platform news, updates & events. Need technical help? Visit https:\/\/twittercommunity.com\/ \u2328\ufe0f #TapIntoTwitter"
  },
  "place": {   
  },
  "entities": {
    "hashtags": [      
    ],
    "urls": [
      {
        "url": "https:\/\/t.co\/XweGngmxlP",
        "unwound": {
          "url": "https:\/\/cards.twitter.com\/cards\/18ce53wgo4h\/3xo1c",
          "title": "Building the Future of the Twitter API Platform"
        }
      }
    ],
    "user_mentions": [     
    ]
  }
}
```

## The Distributed Dream

<img src="img/distributed_dream_graphic.png" alt="document icon" width="800"/>

## The Distributed Reality

<img src="img/cloudformation_fail.png" alt="document icon" width="600"/>

<img src="img/cloudformation_error.png" alt="document icon" width="400"/>

<img src="img/cloudformation_success.png" alt="document icon" width="600"/>
<img src="img/cloudformation_mongo_connection.png" alt="document icon" width="400"/>
<img src="img/cloudformation_mongo_connection_fail.png" alt="document icon" width="400" />

### Problem #1:

<img src="img/cloudformation_cost_estimate.png" alt="document icon" width="500"/>


### Problem #2:

> "Persistent volumes aren't quite 'there' yet. \[...\] it can be dangerous to store data in Kubernetes persistent volumes, since you can't do backup and restore." (Practitioner interview)

[Docker for AWS](https://docs.docker.com/docker-for-aws/)

[MongoDB on the AWS Cloud](https://docs.aws.amazon.com/quickstart/latest/mongodb/welcome.html)

# The Explorations...

* What does a **data model** look like?

* What are some **query examples**--including spatial queries--that show off this database's special niche?

## The Tutorials...

https://github.com/linkalis/nosql_databases_for_spatial_analysis

# MongoDB

<img src="img/mongodb_map.png" alt="Mongodb Compass app" width="800"/>

## What does a data model look like?

Image credit: https://docs.mongodb.com/manual/core/data-model-design/ 


<img src="img/mongo_embedded_data_model.svg" alt="Mongo embedded data model" width="450" style="float:left;"/>

<img src="img/mongo_normalized_data_model.svg" alt="Mongo normalized data model" width="450" style="float:right;"/>

## Query examples

#### Which cities have the most tweets associated with them?

```
db.tweets.aggregate([
    { $match: { 'place.place_type': 'city' } },
    { $group: { _id: { place_id: '$place.id', place_full_name: '$place.full_name' }, tweet_count: { $sum: 1 } } },
    { $sort : { tweet_count : -1 } },
    { $limit: 10 }
])
```



<img src="img/mongodb_aggregation_gui.png" alt="Mongo Compass aggregation GUI" width="800"/>

#### Which countries have the most tweets associated with them?

```
db.tweets.mapReduce(
    function() { emit({ place_id: this.place.id, place_full_name: this.place.full_name }, 1); },    
    function(key, value) { return Array.sum(value); },    
    {
        query: { 'place.place_type': 'country' },
        out: 'tweet_count'
    }
)
```

<img src="img/mongodb_mapreduce_query_results.png" alt="Mongo map reduce results displayed in Compass app" width="800"/>

#### Find all Yelp businesses that are located in Wisconsin, then count the tweets whose bounding box rectangles overlap the business' lat/long coordinates. (Spatial join)

Yelp Dataset Challenge: https://www.yelp.com/dataset/challenge

```
db.yelp_businesses.find({ 'state': {$eq: 'WI'} }).forEach(function(element) {
  var intersecting_tweets = db.tweets.find({
   'place.better_bounding_box': {
      $geoIntersects: {
         $geometry: {
            type: "Point",
            coordinates: [ 
              element.location.coordinates[0],               
              element.location.coordinates[1] 
            ]
         }
      }
   }
  }).count();

  element['intersecting_tweets_count'] = intersecting_tweets;
  db.businesses_tweets_joined.save(element);
});
```

# Neo4j

<img src="img/neo4j.png" alt="Neo4j browser" width="800"/>

## What does a data model look like?

<img src="img/neo4j_data_model.png" alt="graph icon" width="600"/>

emphasize flexability, ability to add arbitrary relationships later on as necessary

```
MERGE (t:Tweet {tweet_id: toInteger(tweet.id_str)})
ON CREATE SET t.text = tweet.text,
	t.timestamp_ms = toInteger(tweet.timestamp_ms),
    ...

MERGE (u:User {user_id: toInteger(tweet.user.id)})
SET	u.name = tweet.user.name,
	u.screen_name = tweet.user.screen_name,
	u.description = tweet.user.description,
	...

CREATE (t)-[:TWEETED_BY]->(u)
CREATE (u)-[:TWEETED]->(t)

MERGE (p:Place {place_id: toString(place_id)})
SET	p.name = tweet.place.name,
	...
	p.bounding_box_LL = point({ 
        longitude: toFloat(tweet.place.bounding_box.coordinates[0][0][0]), 
        latitude: toFloat(tweet.place.bounding_box.coordinates[0][0][1]) 
    }),

CREATE (t)-[:LOCATED_AT]->(p)

WITH t, tweet, tweet.entities.user_mentions AS mentions
UNWIND mentions AS mention
MERGE (mentioned_user:User {id: toInteger(mention.id), name: mention.name, screen_name: mention.screen_name})
CREATE (t)-[:MENTIONS]->(mentioned_user)

WITH t, tweet, tweet.entities.hashtags AS hashtags
UNWIND hashtags AS hashtag
	MERGE (h:Hashtag {hashtag_id: hashtag.text})
	CREATE (t)-[:HASHTAGS]->(h)
```

MERGE helps with uniqueness; will check if a node already exists before creating a new one

UNWIND flattens out values stored in an array

## Query examples

#### Which tweets were tweeted in Minneapolis?

```
MATCH (t:Tweet)-[:LOCATED_AT]->(p:Place)
WHERE p.full_name =~ 'Minneapolis.*'
RETURN t
```

<img src="img/neo4j_text_filter_query_results.png" alt="query results, with node diagram expanded in browser window" width="800"/>

#### Which users appear to be the most influential, based on how many mentions they've received?

```
CALL algo.pageRank.stream('User', 'MENTIONS', { iterations: 20, dampingFactor: 0.85 })
YIELD nodeId, score
MATCH (u) WHERE id(u) = nodeId
RETURN u.screen_name AS user, score
ORDER BY score DESC
```

<img src="img/neo4j_pagerank_query_results.png" alt="page rank query results, with realDonaldTrump at the top" width="800"/>

# Elasticsearch

![Kibana](img/kibana.png)

## What does a data model look like?

```
PUT twitter_sample
{
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "tweet": {
            "_source": { "enabled": false },
            "properties": {
                "text": {"type": "text" },
                "timestamp_ms": { "type": "date", "format": "epoch_millis" },
                "user": { 
                    "properties": { 
                        "location": { "type": "text" },
                        "description": { "type": "text" }
                    }
                },
                "place": { 
                    "properties": { 
                        "name": { "type": "keyword" },
                        "full_name": { "type": "keyword" },
                        "centroid": { "type": "geo_shape" },
                        "better_bounding_box": { "type": "geo_shape" },
                        "centroid_geohash": { "type": "geo_point" }
                    }
                }
            }
        }
    }
}
```

## Query examples

#### What words rhyme with orange?

```
GET twitter_small_with_source/_search
{ "_source": ["id", "text", "user.location", "place.full_name"],
  "query": {
    "bool": {
      "must": {
        "multi_match" : {
            "query": "orange",
            "fields": ["text", "user.location", "place.full_name"],
            "fuzziness": 2
        }
      },
      "must_not": {
        "multi_match" : {
            "query": "orange",
            "fields": ["text", "user.location", "place.full_name"]
        }
      }
    }
  }
}
```

<img src="img/elasticsearch_rhyme_query_results.png" alt="Elasticsearch orange rhyme results" width="600" style="float:left;"/>

# NoSQL Database Smackdown!

<img src="img/big_bang_theory_smackdown.jpg" alt="Big Bang Theory boxing match" width="400"/>

## Load time comparison

These comparisons were run with:
* 500MB of data (~200,000 tweets), spread across 40 files with 5,000 tweets each
* Indexes set on the tweets' "id" field (and in the case of Neo4j, also on user.id and place.id) that enforce a uniqueness constraint

<img src="img/mongodb_bulk_docker_localhost_500MB_load_times.png" alt="MongoDB load times graph" width="300" style="float:left;"/>

<img src="img/neo4j_docker_localhost_500MB_load_times_with_indexes.png" alt="Neo4j load times graph" width="300" style="float:right;"/>

<br>

<img src="img/elasticsearch_bulk_docker_localhost_500MB_load_times.png" alt="Elasticsearch load times graph" width="300" style="float:center;"/>

## Basic query benchmarks

#### How many tweets were tweeted by users with a follower count > 10000?

```
GET twitter_small/_search
{
    "query": {
        "range" : {
            "user.followers_count": {
                "gt" : 10000
            }
        }
    }
}

```
<hr>

```
{ 'user.followers_count': {$gt: 10000} }
```
<hr>

```
MATCH (t:Tweet)-[:TWEETED_BY]->(u:User)
WHERE u.followers_count > 10000
RETURN count(t)
```

<img src="img/query_time_comparison.png" alt="query time comparison" width="600"/>

# I'm ready to say 'no' to SQL! How do I get started?

<img src="img/nosql_decision_chart.png" alt="noSQL decision chart" width="800"/>

<img src="img/oreilly_designing_data_intensive_applications.png" alt="Designing Data-Intensive Applications Book Cover" width="250" style="float:left;padding:10px"/>

<img src="img/nosql_distilled.jpg" alt="NoSQL Distilled Book Cover" width="250" style="float:left;padding:10px"/>

<img src="img/lynda_database_clinic.png" alt="Lynda Database Clinic screenshot" width="300" style="float:left;padding:10px"/>